In [1]:
import pandas as pd
import json
import networkx as nx
import requests
# %pip install scipy==1.8.0

In [51]:
df=pd.read_csv('data/tmdb_5000_credits.csv')
df.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [52]:
cast_list = df.cast.tolist()
# print(cast_list[0])
# print(len(cast_list))
crew_list = df.crew.tolist()
# print(cast_list[0])
# print(len(cast_list))
movie_list = df.movie_id.tolist()
# print(movie_list[0])
# print(len(movie_list))

In [53]:
# Creating a mapping of movie_id to cast and crew
m_mapping={}
for i in range(int(len(cast_list)/10)):
  obj1 = json.loads(cast_list[i])
  obj1 = obj1[:3]
  obj2 = json.loads(crew_list[i])
  crew_set = set()
  for crew_i in obj2:
    if crew_i['job'] == 'Producer' or crew_i['job'] == 'Director' or crew_i['job'] == 'Casting':
      crew_set.add((crew_i['id'],crew_i['name'],crew_i['job']))
  for actor_i in obj1:
      crew_set.add((actor_i['id'],actor_i['name'],'actor'))
  m_mapping[movie_list[i]]=list(crew_set)

In [54]:
temp=[]
for item in m_mapping:
  temp = temp + m_mapping[item]
lst = set(temp)

In [2]:
# Making a call to Celebrity API to get information about celebrities using their name
def api_call(name):
    api_url = 'https://api.api-ninjas.com/v1/celebrity?name={}'.format(name)
    response = requests.get(api_url, headers={'X-Api-Key': 'HZpNjnM9wSBa+dDw0m2EbQ==Rj6TyfZ52NK26AYr'})
    if response.status_code == requests.codes.ok:
        txt = json.loads(response.text)
    else:
        print("Error:", response.status_code, response.text)
    return txt

In [5]:
# Example of a call to the API
txt = api_call("Deepak Chopra")
txt

[{'name': 'deepak chopra',
  'net_worth': 150000000,
  'gender': 'male',
  'nationality': 'us',
  'occupation': ['physician',
   'writer',
   'public_speaker',
   'actor',
   'film_producer',
   'screenwriter'],
  'birthday': '1947-10-22',
  'age': 76,
  'is_alive': True}]

In [45]:
# Creating a graph to map data about celebrities using an api call to Celebrity API
g = nx.Graph()
for item in lst:
    celeb_name = item[1]
    celeb_data = api_call(celeb_name)
    worth = ''
    nationality = ''
    gender = ''
    if celeb_data:
      for i in celeb_data:
        f=0
        try:
          for occ in i['occupation']:
            if item[2].lower() in occ:
              f=1
              break
          if f==1:
            try:
              nationality =i["nationality"]
              worth=i["net_worth"]
              gender=i["gender"] 
              # print(nationality, worth, gender)
            except:
              pass
            break
        except:
            pass
    # print(celeb_name,nationality, worth, gender)
    g.add_node(item[0],name=item[1],job=item[2],nationality=nationality,worth=worth,gender=gender)

In [57]:
def convert_into_dict(temp):
  Dict={}
  for data in temp:
    sorted(temp[data],key=lambda x: x[1])
    for i in range(len(temp[data])-1):
      for j in range(i+1,len(temp[data])):
        if temp[data][i][0]!=temp[data][j][0]:
          if (temp[data][i],temp[data][j]) in Dict.keys():
            Dict[(temp[data][i],temp[data][j])]+=1
          else:
            Dict[(temp[data][i],temp[data][j])]=1
  return Dict

In [58]:
m_mapping_Dict = convert_into_dict(m_mapping)
print(len(m_mapping_Dict))
print(len(lst))
for p in m_mapping_Dict:
  g.add_edge(p[0][0],p[1][0], weight=m_mapping_Dict[p])

14299
1939


In [59]:
# Create subgraphs for each job
subgraph_actors = g.subgraph([a for a in list(g.nodes) if g.nodes[a]['job']=='actor'])
print("Number of Actor Nodes-> ", subgraph_actors.number_of_nodes())

subgraph_producers = g.subgraph([p for p in list(g.nodes) if g.nodes[p]['job']=='Producer'])
print("Number of Producer Nodes-> ",subgraph_producers.number_of_nodes())

subgraph_directors = g.subgraph([d for d in list(g.nodes) if g.nodes[d]['job']=='Director'])
print("Number of Director Nodes-> ",subgraph_directors.number_of_nodes())

subgraph_casting = g.subgraph([c for c in list(g.nodes) if g.nodes[c]['job']=='Casting'])
print("Number of Casting Director Nodes-> ",subgraph_casting.number_of_nodes())

# Print the total number of nodes in the graph
print("Total Number of Nodes-> ",g.number_of_nodes())

Number of Actor Nodes->  705
Number of Producer Nodes->  645
Number of Director Nodes->  234
Number of Casting Director Nodes->  241
Total Number of Nodes->  1825


In [60]:
# Getting the density of the collaborative graph
print("Density of Graph-> ",nx.density(g))

Density of Graph->  0.00785267964431627


In [61]:
# Finding the degree of each node in the graph for each job
g_degree = list(g.degree)

actor_degree=[(g.nodes[a[0]]['name'],a[1]) for a in g_degree if g.nodes[a[0]]['job']=='actor']
actor_degree.sort(key=lambda x: x[1],reverse=True)

producer_degree=[(g.nodes[p[0]]['name'],p[1]) for p in g_degree if g.nodes[p[0]]['job']=='Producer']
producer_degree.sort(key=lambda x: x[1],reverse=True)

director_degree = [(g.nodes[d[0]]['name'],d[1]) for d in g_degree if g.nodes[d[0]]['job']=='Director']
director_degree.sort(key=lambda x: x[1],reverse=True)

casting_degree = [(g.nodes[c[0]]['name'],c[1]) for c in g_degree if g.nodes[c[0]]['job']=='Casting']
casting_degree.sort(key=lambda x: x[1],reverse=True)

In [62]:
print('Degree for Actors->')
actor_degree[:3]

Degree for Actors->


[('Angelina Jolie', 86), ('Arnold Schwarzenegger', 74), ('Johnny Depp', 73)]

In [63]:
print('Degree for Producers->')
producer_degree[:3]

Degree for Producers->


[('Tom Cruise', 79), ('Leonardo DiCaprio', 77), ('Brad Pitt', 75)]

In [64]:
print('Degree for Directors->')
director_degree[:3]

Degree for Directors->


[('Ridley Scott', 48), ('Tim Burton', 44), ('Steven Spielberg', 43)]

In [65]:
print('Degree for Casting Directors->')
casting_degree[:3]

Degree for Casting Directors->


[('Lucinda Syson', 113), ('Sarah Finn', 112), ('John Papsidera', 103)]

In [66]:
# Finding the closeness centrality of each node in the graph for each job
g_closeness = nx.closeness_centrality(g)

g_closeness = list(g_closeness.items())

actor_closeness=[(g.nodes[a[0]]['name'],a[1]) for a in g_closeness if g.nodes[a[0]]['job']=='actor']
actor_closeness.sort(key=lambda x: x[1],reverse=True)

producer_closeness=[(g.nodes[p[0]]['name'],p[1]) for p in g_closeness if g.nodes[p[0]]['job']=='Producer']
producer_closeness.sort(key=lambda x: x[1],reverse=True)

director_closeness=[(g.nodes[d[0]]['name'],d[1]) for d in g_closeness if g.nodes[d[0]]['job']=='Director']
director_closeness.sort(key=lambda x: x[1],reverse=True)

casting_closeness=[(g.nodes[c[0]]['name'],c[1]) for c in g_closeness if g.nodes[c[0]]['job']=='Casting']
casting_closeness.sort(key=lambda x: x[1],reverse=True)

In [67]:
print('Closeness for Actors->')
actor_closeness[:3]

Closeness for Actors->


[('Angelina Jolie', 0.39221533554025845),
 ('Johnny Depp', 0.3737970867170089),
 ('Gary Oldman', 0.3686907871332547)]

In [68]:
print('Closeness for Producers->')
producer_closeness[:3]

Closeness for Producers->


[('Brad Pitt', 0.3842750473235369),
 ('Tom Cruise', 0.3796299643339117),
 ('Brian Grazer', 0.3778858756769412)]

In [69]:
print('Closeness for Directors->')
director_closeness[:3]

Closeness for Directors->


[('Steven Spielberg', 0.36464351651241256),
 ('Ridley Scott', 0.3635690039400754),
 ('Jan de Bont', 0.3500134422936775)]

In [70]:
print('Closeness for Casting Directors->')
casting_closeness[:3]

Closeness for Casting Directors->


[('Lucinda Syson', 0.40386166418501246),
 ('Sarah Finn', 0.39248269432385785),
 ('Ronna Kress', 0.38651070434533413)]

In [71]:
# Finding the betweenness centrality of each node in the graph for each job
g_betweenness = nx.betweenness_centrality(g)
g_betweenness = list(g_betweenness.items())

actor_betweenness=[(g.nodes[a[0]]['name'],a[1]) for a in g_betweenness if g.nodes[a[0]]['job']=='actor']
actor_betweenness.sort(key=lambda x: x[1],reverse=True)

producer_betweenness=[(g.nodes[p[0]]['name'],p[1]) for p in g_betweenness if g.nodes[p[0]]['job']=='Producer']
producer_betweenness.sort(key=lambda x: x[1],reverse=True)

director_betweenness=[(g.nodes[d[0]]['name'],d[1]) for d in g_betweenness if g.nodes[d[0]]['job']=='Director']
director_betweenness.sort(key=lambda x: x[1],reverse=True)

casting_betweenness=[(g.nodes[c[0]]['name'],c[1]) for c in g_betweenness if g.nodes[c[0]]['job']=='Casting']
casting_betweenness.sort(key=lambda x: x[1],reverse=True)

In [72]:
print('Betweenness for Actors->')
actor_betweenness[:3]

Betweenness for Actors->


[('Arnold Schwarzenegger', 0.028147915880146313),
 ('Angelina Jolie', 0.02664458433682943),
 ('Hugh Jackman', 0.017077584236605502)]

In [73]:
print('Betweenness for Producers->')
producer_betweenness[:3]

Betweenness for Producers->


[('Tom Hanks', 0.03110445522495556),
 ('Tom Cruise', 0.022563294670007717),
 ('Brad Pitt', 0.021984697846443843)]

In [74]:
print('Betweenness for Directors->')
director_betweenness[:3]

Betweenness for Directors->


[('Ridley Scott', 0.011246156997316837),
 ('Steven Spielberg', 0.007576609567366789),
 ('Tim Burton', 0.005578533063728941)]

In [75]:
print('Betweenness for Casting Directors->')
casting_betweenness[:3]

Betweenness for Casting Directors->


[('Mary Hidalgo', 0.050085236006449034),
 ('Lucinda Syson', 0.040871030304190814),
 ('Sarah Finn', 0.03589732224501019)]

In [76]:
# Finding the clustering coefficient of each node in the graph for each job
g_clustering=nx.clustering(g)
g_clustering=list(g_clustering.items())

actor_clustering = [(g.nodes[a[0]]['name'],a[1]) for a in g_clustering if g.nodes[a[0]]['job']=='actor' and int(a[1])!=1]
actor_clustering.sort(key=lambda x: x[1],reverse=True)

producer_clustering = [(g.nodes[p[0]]['name'],p[1]) for p in g_clustering if g.nodes[p[0]]['job']=='Producer' and int(p[1])!=1]
producer_clustering.sort(key=lambda x: x[1],reverse=True)

director_clustering=[(g.nodes[d[0]]['name'],d[1]) for d in g_clustering if g.nodes[d[0]]['job']=='Director' and int(d[1])!=1]
director_clustering.sort(key=lambda x: x[1],reverse=True)

casting_clustering=[(g.nodes[c[0]]['name'],c[1]) for c in g_clustering if g.nodes[c[0]]['job']=='Casting' and int(c[1])!=1]
casting_clustering.sort(key=lambda x: x[1],reverse=True)


In [82]:
print('Clustering for Actors')
actor_clustering[:3]

Clustering for Actors


[('Tom Hiddleston', 0.9642857142857143),
 ('Josh Duhamel', 0.9454545454545454),
 ('Shailene Woodley', 0.9090909090909091)]

In [81]:
print('Clustering for Producers')
producer_clustering[:3]

Clustering for Producers


[('Nina Jacobson', 0.9523809523809523),
 ('Bill Miller', 0.9523809523809523),
 ('Dan Lin', 0.9454545454545454)]

In [80]:
print('Clustering for Directors')
director_clustering[:3]

Clustering for Directors


[('Guy Ritchie', 0.9454545454545454),
 ('Steven Soderbergh', 0.9333333333333333),
 ('Anthony Russo', 0.9166666666666666)]

In [79]:
print('Clustering for Casting Directors->')
casting_clustering[:3]

Clustering for Casting Directors->


[('Robin Gurland', 0.9333333333333333),
 ('Marianne Stanicheva', 0.9272727272727272),
 ('Venus Kanani', 0.9090909090909091)]